In [1]:
import os, sys
sys.path.append("/home/alexanderalbizu")
sys.path.append("/home/alexanderalbizu/.local/bin")
os.environ["WANDB_NOTEBOOK_NAME"] = "CNN.ipynb"
#!python setup.py develop 
# !pip install wandb
# !pip install 'monai[all]'
#!pip -q install vit_pytorch
#!pip -q install linformer

In [2]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import glob, math, os, shutil, tempfile, time, monai, torch, random

import wandb as wb
from enum import Enum
import pandas as pd
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from monai.networks.utils import eval_mode
from monai.config import print_config
from monai.data import (
    CacheDataset,
    DataLoader,
    ThreadDataLoader,
    ImageDataset,
    Dataset,
    decollate_batch,
)
from sklearn.metrics import (
    classification_report, confusion_matrix,
    ConfusionMatrixDisplay
)
# from vit_pytorch.efficient import ViT
# from linformer import Linformer
from sklearn.model_selection import train_test_split
from monai.inferers import sliding_window_inference
from monai.losses import DiceLoss, DiceCELoss
from monai.metrics import DiceMetric
from monai.networks.layers import Act, Norm
from monai.transforms import (
    AddChannel,
    Compose,
    RandGaussianNoise,
    Resize,
    AsChannelFirst,
    RemoveRepeatedChannel,
    Orientation,
    RandRotate90,
    RandBiasField,
    ScaleIntensity,
    ToDevice,
    EnsureType,
)
from monai.utils import set_determinism

pin_memory = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print_config()
wb.login(); # 7e5f63e5846f29b034d98806712ab047df76834d

MONAI version: 0.8.1+331.g6a301d51.dirty
Numpy version: 1.20.1
Pytorch version: 1.9.0+cu111
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 6a301d51fbbb1803b7349a85c9bfa398f19ee0f9
MONAI __file__: /home/alexanderalbizu/MONAI/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.8
Nibabel version: 3.2.1
scikit-image version: 0.19.1
Pillow version: 9.1.1
Tensorboard version: 1.15.0+nv
gdown version: 4.4.0
TorchVision version: 0.10.0+cu111
tqdm version: 4.53.0
lmdb version: 1.2.1
psutil version: 5.8.0
pandas version: 1.1.4
einops version: 0.4.1
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: 1.26.1
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



wandb: Currently logged in as: aalbizu. Use `wandb login --relogin` to force relogin


In [ ]:
# Set data directory
rootDir = '/blue/camctrp/working/Alejandro/StimBrain/'
batch_size=3
im_size = (96,96,96)
# pat_size = (24,24,24)
grad_clip = 2
step_size = 1
gamma = 7e-1
seed = 42
lr = 1e-2
wd = 0
K = 7 # MUST BE A FACTOR OF NUM SUBS
val_interval = 1
epoch_loss_values = [] # Pre-Allocate
epoch_acc_values = [] # Pre-Allocate
max_epochs = 50
subs = np.array([9009, 9015, 9022, 9040, 9044, 9045, 9051, 9021, 9023, 9031, 9032, 9047, 9048, 9054]) # Subjects
beh = np.array([32, 26, 19, 20, 20, 17, 15, 2, 4, 6, 10, 16, 18, 7]) # Targeted Behavior Change
num_cores = int(os.environ["SLURM_CPUS_PER_TASK"]);

# Define Group Labels as above/below Median
lab = np.int64(np.zeros([len(beh),1]));
lab[beh >= np.median(beh)] = 1; # Responders
lab[beh < np.median(beh)] = 0; # Non Responders

wb.init(project="SMART-CNN",
        config={
       "batch_size": batch_size,
       "n_channels": 3,
       "n_epoch": max_epochs,
       "image_size": im_size,
       "folds": K,
       "gradient_clip": grad_clip,
       "learning_rate": lr,
       "step_size": step_size,
       "network": "ResNet10",
       "gamma": gamma,
       "weight_decay": wd,
       "dataset": "SB",
        })
run_id = wb.run.name;
    
class Diagnosis(Enum):
    NonResponder = 0
    Responder = 1
    
images = np.array([os.path.join(rootDir,''.join(['FS6.0_sub-',str(subs[s]),'_ses01']),'T1_xyzJbrain.nii') for s in range(subs.shape[0])]);

# Plot Labels
# plt.imshow(lab); plt.axis('off'); plt.title('Class Labels');

# def seed_everything(seed):
#     random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed)
#     torch.backends.cudnn.deterministic = True
    
# seed_everything(seed)    

def save_model(n_epoch, save_path, run_id, f):
    lastmodel = f"{save_path}-{run_id}-fold{f}.pth"
    torch.save(
        {
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "best_valid_score": best_valid_score,
            "n_epoch": n_epoch,
        },
        lastmodel,
    )

butt = [0,1];
testIdx = np.array(random.sample(list(np.arange(subs.shape[0])),subs.shape[0]));
alltargs = []; allpreds = []; allscores = []; # Pre-Allocate
for f in range(K):
    print("Beginning Fold ", f+1)
    test_subs = subs[testIdx[butt]]; print('test subs: ', test_subs)
    test_list = images[testIdx[butt]]; 
    test_label = lab[testIdx[butt]];
    nontest_list = images[np.logical_not(np.isin(subs,test_subs))]
    y_nontest = lab[np.logical_not(np.isin(subs,test_subs))]
    train_list, valid_list, train_label, valid_label = train_test_split(nontest_list, y_nontest, test_size=0.10, stratify=y_nontest, random_state=seed);
    
    train_list = np.array(train_list); train_label = np.array(train_label);                  
    valid_list = np.array(valid_list); valid_label = np.array(valid_label);                  
    test_list = np.array(test_list); test_label = np.array(test_label);
    
# print(images.shape, lab.shape)
# nontest_list, test_list = train_test_split(images, 
#                                           test_size=0.1,
#                                           stratify=lab.T,
#                                           random_state=seed)
# nontest_label, test_label = train_test_split(lab.T, 
#                                           test_size=0.1,
#                                           stratify=lab.T,
#                                           random_state=seed)
# train_list, valid_list = train_test_split(nontest_list, 
#                                           test_size=0.3,
#                                           stratify=nontest_label,
#                                           random_state=seed)
# train_label, valid_label = train_test_split(nontest_label, 
#                                           test_size=0.3,
#                                           stratify=nontest_label,
#                                           random_state=seed)

# Oversampling the Unbalnaced Class
# train_list = np.array(np.hstack([[train_list.T],[train_list.T]]))[0,:];
# train_label = np.array([np.int64(np.hstack([train_label[:,0],train_label[:,0]]))]).T;

# print(train_list.shape,valid_list.shape,test_list.shape)
# print(train_label.shape,valid_label.shape,test_label.shape)

    print('train case split: ',sum(train_label)[0],':',len(train_label)-sum(train_label)[0])
    print('valid case split: ',sum(valid_label)[0],':',len(valid_label)-sum(valid_label)[0])
    print('test case split: ',sum(test_label)[0],':',len(test_label)-sum(test_label)[0],'\n')
    del nontest_list, y_nontest; # Save RAM

# Plot Responder Mean
# plt.rcParams['figure.figsize'] = [5,5];
# print(images[0])
# nii = nib.load(images[0]); print(nii.shape) # Load Each Electrode
# plt.imshow(nii.get_fdata()[:, :, 120,0].T, cmap="turbo", origin="lower"); plt.axis('off'); # Plot

# Represent labels in one-hot format for binary classifier training,
# BCEWithLogitsLoss requires target to have same shape as input
#     labels = torch.nn.functional.one_hot(torch.as_tensor(lab.T)).long();
    train_lab = torch.nn.functional.one_hot(torch.as_tensor(train_label.T)).long(); 
    valid_lab = torch.nn.functional.one_hot(torch.as_tensor(valid_label.T)).long();
    test_lab = torch.nn.functional.one_hot(torch.as_tensor(test_label.T)).long();

    # Define transforms
    train_transforms = Compose([
    #     ScaleIntensity(minv=0.0, maxv=1.0),
        AsChannelFirst(channel_dim=3),
    #     AddChannel(),
        Resize(im_size),
    #     RandBiasField(),
    #     RandGaussianNoise(prob=0.1), 
        EnsureType(data_type='tensor')]);

    val_transforms = Compose([
    #     ScaleIntensity(minv=0.0, maxv=1.0),
        AsChannelFirst(channel_dim=3),
    #     AddChannel(),
        Resize(im_size),
        EnsureType(data_type='tensor')]);

    # Define nifti dataset, data loader
    check_ds = ImageDataset(image_files=train_list, labels=train_lab[0,:,:], transform=train_transforms);
    check_loader = DataLoader(check_ds, batch_size=1, num_workers=num_cores, pin_memory=pin_memory); 

    im, label = monai.utils.misc.first(check_loader); print(im.shape, label, label.shape)
    plt.imshow(im[0,0,:,:,60].T, cmap="turbo", origin="lower"); plt.axis('off'); del check_ds, check_loader, im, label;

    # create a training data loader
    train_ds = ImageDataset(image_files=train_list, labels=train_lab[0,:,:], transform=train_transforms);
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=num_cores, pin_memory=pin_memory);

    # create a validation data loader
    val_ds = ImageDataset(image_files=valid_list, labels=valid_lab[0,:,:], transform=val_transforms);
    val_loader = DataLoader(val_ds, batch_size=batch_size, num_workers=num_cores, pin_memory=pin_memory)

    # Create DenseNet121, CrossEntropyLoss and Adam optimizer
    # model = monai.networks.nets.DenseNet121(spatial_dims=3, in_channels=3, out_channels=2).to(device)
    model = monai.networks.nets.resnet10(spatial_dims=3, num_classes=2).to(device)

    # Loss Function
    loss_fx = torch.nn.BCEWithLogitsLoss() 

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

    wb.watch(model, log='all')

    scaler = torch.cuda.amp.GradScaler()
    best_valid_score = 99999; # Initialize Loss
    lastmodel = None
    for epoch in range(max_epochs):
        epoch_loss = 0
        epoch_accuracy = 0

        for batch_data in train_loader:
            inputs, label = batch_data[0].to(device), batch_data[1].to(device);

            # Evaluate Model
            output = model(inputs);
            loss = loss_fx(output, label.float())

            # Update Gradient
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                # Evaluate Model
                output = model(inputs); epoch_prob = torch.nn.functional.softmax(output,dim=1)
                loss = loss_fx(output, label.float())
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            # Accuracy
            acc = (torch.nn.functional.softmax(epoch_prob,dim=1).argmax(dim=1) == label.argmax(dim=1)).float().mean()
            epoch_accuracy += acc / len(train_loader)
            epoch_acc_values.append(acc)

            # Loss
            epoch_loss += loss / len(train_loader)
            epoch_loss_values.append(epoch_loss)
    #         print(f"train loss: {loss.item():.4f}")
            wb.log({'train_loss': loss, 'train_acc': acc})

        if epoch % val_interval == 0: # Validation Interval
            with eval_mode(model):
                epoch_val_accuracy = 0; epoch_val_loss = 0;
                for val_data in val_loader:
                    val_images, val_labels = val_data[0].to(device), val_data[1].to(device)

                    val_output = model(val_images); val_prob = torch.nn.functional.softmax(val_output,dim=1);
                    val_loss = loss_fx(val_output, val_labels.float())

                    val_acc = (val_prob.argmax(dim=1) == val_labels.argmax(dim=1)).float().mean()
                    epoch_val_accuracy += val_acc / len(val_loader)
                    epoch_val_loss += val_loss / len(val_loader)
                    wb.log({'val_loss': val_loss, 'val_acc': val_acc})
            print(
                f"Epoch : {epoch+1} - train_loss : {epoch_loss:.4f} - train_acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
            )

            # Save Best Model
            if best_valid_score > epoch_val_loss:
                print("model saved")
                save_model(epoch, "T1", run_id,f)
                best_valid_score = epoch_val_loss

    #     else:    
    #         print(
    #             f"Epoch : {epoch+1} - train_loss : {epoch_loss:.4f} - train_acc: {epoch_accuracy:.4f} \n"
    #         )

#     wb.run.log_code(root=os.path.join(os.getcwd(),"CNN_SMART.ipynb"));
    
    ##############
    # Test Model #
    ##############
    
    modelfile = f'T1-'+run_id+'.pth'; # HARDCODED
    checkpoint = torch.load(modelfile)
    print(f"Loading {modelfile} ...")

    # create a validation data loader
    test_ds = ImageDataset(image_files=test_list, labels=test_lab[0,:,:], transform=val_transforms)
    test_loader = DataLoader(test_ds, batch_size=batch_size, num_workers=num_cores, pin_memory=pin_memory)
    model.load_state_dict(checkpoint["model_state_dict"])

    with eval_mode(model):
        y_pred = torch.tensor([], dtype=torch.float32, device=device)
        y = torch.tensor([], dtype=torch.long, device=device)
        test_acc = 0
        for test_data in test_loader:
            test_images, test_labels = test_data[0].to(
                device), test_data[1].to(device)

            outputs = model(test_images); probs = torch.nn.functional.softmax(outputs,dim=1);
            test_acc += (probs.argmax(dim=1) == test_labels.argmax(dim=1)).float().mean();
            y_pred = torch.cat([y_pred, probs.argmax(dim=1)], dim=0)
            y = torch.cat([y, test_labels.argmax(dim=1)], dim=0);
        test_acc = test_acc / len(test_loader)
        alltargs.append(y.cpu().numpy())
        allpreds.append(y_pred.cpu().numpy())
        allscores.append(y_prob.cpu().numpy())
        print(f"Fold {f+1} Test Accuracy: ", test_acc.cpu().numpy())
        wb.log({'test_acc': test_acc})
    butt = [ x + 2 for x in butt ]
        
print(classification_report(
    alltargs,
    allpreds,
    target_names=[d.name for d in Diagnosis]))

cm = confusion_matrix(
    alltargs,
    allpreds,
    normalize='true')

disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=[d.name for d in Diagnosis])
disp.plot(ax=plt.subplots(1, 1, facecolor='white')[1])

wb.log({"conf_mat" : wb.plot.confusion_matrix(probs=None,
                        y_true=alltargs, preds=allpreds,
                        class_names=Diagnosis)})
wb.finish()

Beginning Fold  1
test subs:  [9051 9023]
train case split:  6 : 4
valid case split:  1 : 1
test case split:  0 : 2 



Modifying image pixdim from [1. 1. 1. 1.] to [  1.           1.           1.         167.79079236]
Modifying image pixdim from [1. 1. 1. 1.] to [  1.           1.           1.         167.79079236]
Modifying image pixdim from [1. 1. 1. 1.] to [  1.           1.           1.         167.79079236]
Modifying image pixdim from [1. 1. 1. 1.] to [  1.           1.           1.         167.79079236]


torch.Size([1, 3, 96, 96, 96]) tensor([[1, 0]]) torch.Size([1, 2])


Modifying image pixdim from [1. 1. 1. 1.] to [  1.           1.           1.         167.79079236]
Modifying image pixdim from [1. 1. 1. 1.] to [  1.           1.           1.         167.79079236]
Modifying image pixdim from [1. 1. 1. 1.] to [  1.           1.           1.         167.79079236]
Modifying image pixdim from [1. 1. 1. 1.] to [  1.           1.           1.         167.79079236]
Modifying image pixdim from [1. 1. 1. 1.] to [  1.           1.           1.         167.79079236]


Epoch : 1 - train_loss : 1.1418 - train_acc: 0.2500 - val_loss : 38.3673 - val_acc: 0.5000

model saved


Modifying image pixdim from [1. 1. 1. 1.] to [  1.           1.           1.         167.79079236]
Modifying image pixdim from [1. 1. 1. 1.] to [  1.           1.           1.         167.79079236]
Modifying image pixdim from [1. 1. 1. 1.] to [  1.           1.           1.         167.79079236]
Modifying image pixdim from [1. 1. 1. 1.] to [  1.           1.           1.         167.79079236]
Modifying image pixdim from [1. 1. 1. 1.] to [  1.           1.           1.         167.79079236]


Epoch : 2 - train_loss : 1.3713 - train_acc: 0.3333 - val_loss : 13.1579 - val_acc: 0.5000

model saved


Modifying image pixdim from [1. 1. 1. 1.] to [  1.           1.           1.         167.79079236]


In [ ]:
# cam = monai.visualize.CAM(nn_module=model_3d, target_layers="class_layers.relu", fc_layers="class_layers.out")
cam = monai.visualize.GradCAMpp(
    nn_module=model, target_layers="class_layers.relu"
)
# cam = monai.visualize.GradCAMpp(nn_module=model_3d, target_layers="class_layers.relu")
print(
    "original feature shape",
    cam.feature_map_size([1, 1] + list(im_size), device),
)
print("upsampled feature shape", [1, 1] + list(im_size))

occ_sens = monai.visualize.OcclusionSensitivity(
    nn_module=model, mask_size=12, n_batch=1, stride=24
)

# For occlusion sensitivity, inference must be run many times. Hence, we can use a
# bounding box to limit it to a 2D plane of interest (z=the_slice) where each of
# the arguments are the min and max for each of the dimensions (in this case CHWD).
the_slice = train_ds[0][0].shape[-1] // 2
occ_sens_b_box = [-1, -1, -1, -1, -1, -1, the_slice, the_slice]

train_transforms.set_random_state(42)
n_examples = 3
subplot_shape = [3, n_examples]
fig, axes = plt.subplots(*subplot_shape, figsize=(25, 15), facecolor="white")
items = np.random.choice(len(train_ds), size=len(train_ds), replace=False)

example = 0
for item in items:

    data = train_ds[
        item
    ]  # this fetches training data with random augmentations
    image, label = data[0].to(device).unsqueeze(0), data[1][1]
    y_pred = model(image); prob = torch.nn.functional.softmax(y_pred,dim=1);
    pred_label = prob.argmax(dim=1);
    
    # Only display preMCI
    if np.not_equal(label,1):
        continue

    img = image.detach().cpu().numpy()[..., the_slice]

    name = "actual: "
    name += "Responder" if np.equal(label,1) else "Non-Responder"
    name += "\npred: "
    name += "Responder" if np.equal(pred_label.cpu().numpy(),1) else "Non-Responder"
    name += f"\nResponder: {prob[0,1]:.3}"
    name += f"\nNon-Responder: {prob[0,0]:.3}"

    # run CAM
    cam_result = cam(x=image, class_idx=None)
    cam_result = cam_result[..., the_slice]

    # run occlusion
    occ_result, _ = occ_sens(x=image, b_box=occ_sens_b_box)
    occ_result = occ_result[..., pred_label];

    for row, (im, title) in enumerate(
        zip(
            [img, cam_result, occ_result],
            [name, "CAM", "Occ. Sens."],
        )
    ):
        cmap = "gray" if np.equal(row,0) else "jet"
        ax = axes[row, example]
        if isinstance(im, torch.Tensor):
            im = im.cpu().detach()
        
        im_show = ax.imshow(np.squeeze(im[0][0].T), cmap=cmap, origin='lower')

        ax.set_title(title, fontsize=25)
        ax.axis("off")

    example += 1
    if example == n_examples:
        break